## Setting up the notebook

High-level configs

In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv

# Load environment variables from .env file. Adjust the path to the .env file as needed.
load_dotenv(dotenv_path='../.env')

# Enable asyncio in Jupyter
import asyncio
import nest_asyncio

nest_asyncio.apply()

#  Add the package to the path (required if you are running this notebook from the examples folder)
import sys
sys.path.append('../../')


Import required packages

In [6]:
import json
import pandas as pd
from pydantic import BaseModel
from tqdm.auto import tqdm

from lattereview.providers import OpenAIProvider
from lattereview.providers import LiteLLMProvider
from lattereview.agents import AbstractionReviewer
from lattereview.workflows import ReviewWorkflow

## Data

Building five example stories and dummy question-answering pairs from each story:

In [3]:
class BuildStoryOutput(BaseModel):
    story: str
    location: str
    characters: list[str]

async def build_story():
    prompt = """
    Write a one-paragraph story with whatever realistic or imaginary theme you like,  
    then create a list of all characters you named in your story.
    Return your story, the main location that your story happens in, and a Python list of your characters as your output.
    """
    provider = OpenAIProvider(model="gpt-4o", response_format_class=BuildStoryOutput)
    return await provider.get_json_response(prompt, temperature=0.9)

def run_build_story():
    response =  asyncio.run(build_story())[0]
    return response

data = {
    "story": [],
    "location": [],
    "characters": [],
}
for i in tqdm(range(5)):
    out = json.loads(run_build_story())
    data["characters"].append(out["characters"])
    data["location"].append(out["location"])
    data["story"].append(out["story"])


data = pd.DataFrame(data)
data.to_csv("data.csv", index=False)
data

100%|██████████| 5/5 [00:13<00:00,  2.61s/it]


,story,location,characters
0,"In the mystical town of Elderia, nestled deep ...",Elderia,"[Alaric, Luminara]"
1,"On the edge of the serene Lake Eldoria, the ec...",Lake Eldoria,"[Milo Gearston, Elara, Oliver]"
2,"In the heart of the ancient, whispering forest...",Eldergreen Forest,"[Dr. Mabel Tintwhistle, Whiskers, Simon Sprocket]"
3,"In the heart of the Enchanted Forest, where th...",Enchanted Forest,"[Elara, Finn, Orin]"
4,"In the heart of the Verdant Wood, a magical fo...",Verdant Wood,"[Lyra, Oliver]"


## Abstraction with a single agent

In [4]:
Albert = AbstractionReviewer(
    provider=LiteLLMProvider(model="gpt-4o-mini"),
    name="Albert",
    max_concurrent_requests=1, 
    backstory="an expert reviewer!",
    input_description = "stories",
    model_args={"max_tokens": 200, "temperature": 0.1},
    abstraction_keys = {
        "location": str, 
        "characters": list[str]
    },
    key_descriptions = {
        "location": "The main location that the story happens in.", 
        "characters": "The name of the characters mentioned in the story."
    }
)


# Dummy input
input_list = data.story.str.lower().tolist()
print("====== Inputs ======\n\n", '\n'.join(input_list))

# Dummy review
results, total_cost = asyncio.run(Albert.review_items(input_list))
print("\n====== Outputs ======")
for result in results:
    print(result)

# Dummy costs
print("\n====== Costs ======\n")
for i, item in enumerate(Albert.memory):
    print(f"Cost for item {i}: {item['cost']}")

print(f"\nTotal cost: {total_cost}")

====== Inputs ======

 in the mystical town of elderia, nestled deep within the whispering pines, lived an eccentric inventor named alaric. alaric spent his days and nights in his cluttered workshop, tinkering with peculiar gadgets and magical contraptions. one rainy evening, as alaric was experimenting with a device designed to capture moonbeams, he accidentally summoned a sprite named luminara. the tiny, glowing creature fluttered around the workshop, bringing to life alaric's collection of automaton birds. confused but intrigued, alaric and luminara struck a friendship, promising adventures that would unravel the hidden secrets of elderia.
on the edge of the serene lake eldoria, the eccentric inventor milo gearston unveiled his latest creation: a clockwork bird that sang like the morning sun. as the townspeople gathered to witness this marvel, young elara, the curious daughter of the mayor, tiptoed closer to get a better view. suddenly, the bird sprang to life, its gears whirring ge

Reviewing 5 items - 2024-12-28 14:32:02: 100%|██████████| 5/5 [00:05<00:00,  1.02s/it]


====== Outputs ======
{'location': 'the mystical town of elderia', 'characters': ['alaric', 'luminara']}
{'location': 'the serene lake Eldoria', 'characters': ['Milo Gearston', 'Elara', 'Oliver']}
{'location': 'the heart of the ancient, whispering forest of eldergreen', 'characters': ['dr. mabel tintwhistle', 'whiskers', 'simon sprocket']}
{'location': 'the heart of the enchanted forest', 'characters': ['elara', 'finn', 'orin']}
{'location': 'the heart of the verdant wood, a magical forest', 'characters': ['lyra', 'oliver']}

====== Costs ======

Cost for item 0: 6.434999999999999e-05
Cost for item 1: 7.11e-05
Cost for item 2: 7.965e-05
Cost for item 3: 7.065e-05
Cost for item 4: 7.035e-05

Total cost: 7.035e-05


## Abstraction with a workflow

In [5]:
workflow = ReviewWorkflow(
    workflow_schema=[
        {
            "round": 'A',
            "reviewers": [Albert],
            "text_inputs": ["story"]
        }
    ]
)

# Reload the data if needed.
updated_data = asyncio.run(workflow(data))

print("\n====== Costs ======\n")
print("Total cost: ", workflow.get_total_cost())
print("Detailed costs: ", workflow.reviewer_costs)

updated_data


====== Starting review round A (1/1) ======

Processing 5 eligible rows


['round: A', 'reviewer_name: Albert'] -                     2024-12-28 14:32:07:   0%|          | 0/5 [00:00<?, ?it/s]

['round: A', 'reviewer_name: Albert'] -                     2024-12-28 14:32:07: 100%|██████████| 5/5 [00:04<00:00,  1.14it/s]

The following columns are present in the dataframe at the end of Albert's reivew in round A: ['story', 'location', 'characters', 'round-A_Albert_output', 'round-A_Albert_location', 'round-A_Albert_characters']

====== Costs ======

Total cost:  6.63e-05
Detailed costs:  {('A', 'Albert'): 6.63e-05}


,story,location,characters,round-A_Albert_output,round-A_Albert_location,round-A_Albert_characters
0,"In the mystical town of Elderia, nestled deep ...",Elderia,"[Alaric, Luminara]","{'location': 'Elderia', 'characters': ['Alaric...",Elderia,"[Alaric, Luminara]"
1,"On the edge of the serene Lake Eldoria, the ec...",Lake Eldoria,"[Milo Gearston, Elara, Oliver]","{'location': 'Lake Eldoria', 'characters': ['M...",Lake Eldoria,"[Milo Gearston, Elara, Oliver]"
2,"In the heart of the ancient, whispering forest...",Eldergreen Forest,"[Dr. Mabel Tintwhistle, Whiskers, Simon Sprocket]","{'location': 'the ancient, whispering forest o...","the ancient, whispering forest of Eldergreen","[Dr. Mabel Tintwhistle, Whiskers, Simon Sprocket]"
3,"In the heart of the Enchanted Forest, where th...",Enchanted Forest,"[Elara, Finn, Orin]","{'location': 'Enchanted Forest', 'characters':...",Enchanted Forest,"[Elara, Finn, Orin]"
4,"In the heart of the Verdant Wood, a magical fo...",Verdant Wood,"[Lyra, Oliver]","{'location': 'Verdant Wood', 'characters': ['L...",Verdant Wood,"[Lyra, Oliver]"
